<a href="https://colab.research.google.com/github/mille055/Rec_Project/blob/main/notebooks/exploringdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import os
import numpy as np
import pandas as pd
import string
import time
import urllib.request
import zipfile
import torch
import json
import pickle
from google.colab import files
from google.colab import drive

import zipfile
import time

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt


from sklearn.linear_model import LogisticRegression



In [2]:
### mount google drive

drive.mount('/content/gdrive/', force_remount=True)


Mounted at /content/gdrive/


In [3]:
# Unzip the data
datapath = '/content/gdrive/MyDrive/AIPI540_RecProject/'

if not os.path.exists('./data'):
    os.makedirs('./data')

zip_ref = zipfile.ZipFile(datapath+'reviews.json.zip', 'r')
zip_ref.extractall('data/')
zip_ref.close()

In [4]:
zip_ref = zipfile.ZipFile(datapath+'podcasts.json.zip', 'r')
zip_ref.extractall('data/')
zip_ref.close()

In [5]:
zip_ref = zipfile.ZipFile(datapath+'categories.json.zip', 'r')
zip_ref.extractall('data/')
zip_ref.close()

In [ ]:
# podcasts_df = pd.read_json('data/podcasts.json', lines=True)
# categories_df = pd.read_json('data/categories.json', lines=True)
# reviews_df = pd.read_json('data/reviews.json', lines=True)

In [6]:
with open('data/categories.json') as f:
    categories_df = pd.DataFrame(json.loads(line) for line in f)
categories_df.head()

,podcast_id,itunes_id,category
0,943d1c4349913bde09ef82b69f83cfdf,1446474395,education
1,f53f27d03cdc5040ab4fd2ea30e3dd4f,840646770,society-culture
2,49fa5f1097ae9046ce1a0c56e31b533c,1541959455,arts-performing-arts
3,dbe2dc5aa097523aa294a3b659293af4,1146609744,tv-film
4,2349ed9d26f16029cfc69902bf319ca5,1475252651,education


In [7]:
with open('data/podcasts.json') as f:
    podcasts_df = pd.DataFrame(json.loads(line) for line in f)
podcasts_df.head()

,podcast_id,itunes_id,slug,itunes_url,title,author,description,average_rating,ratings_count,scraped_at
0,d3997f1089be9e45fda4113be39765cb,1000000618,christianity-questions-and-answers,https://podcasts.apple.com/us/podcast/christia...,None,None,None,NaN,None,None
1,f201172864310534ae6b6c4f8ae9b8cc,1000016089,rahdo-talks-through,https://podcasts.apple.com/us/podcast/rahdo-ta...,Rahdo Talks Through,noreply@blogger.com (Richard Ham),"Richard ""Rahdo"" Ham talks (at great length) al...",4.6,60,2019-07-08 06:15:45.106345
2,4e5ce6a47e5d491aac3f588cfb3ece73,1000016800,st-neots-evangelical-church-sermons,https://podcasts.apple.com/us/podcast/st-neots...,None,None,None,NaN,None,None
3,6c476d3dd90c3fe5381153354b326952,1000035657,idiotspeakshow,https://podcasts.apple.com/us/podcast/idiotspe...,IdiotSpeakShow,IdiotSpeakShow,Podcast by IdiotSpeakShow,NaN,None,2019-07-08 06:01:23.683147
4,b9e7fdf5cd545fc8097055e2f2f1d760,1000035702,conciertos-en-el-living,https://podcasts.apple.com/us/podcast/conciert...,None,None,None,NaN,None,None


In [20]:
podcasts_limited = podcasts_df[['podcast_id', 'itunes_id', 'author', 'slug', 'title', 'description', 'average_rating', 'ratings_count']]
podcasts_limited = podcasts_limited.dropna()
podcasts_limited

,podcast_id,itunes_id,author,slug,title,description,average_rating,ratings_count
1,f201172864310534ae6b6c4f8ae9b8cc,1000016089,noreply@blogger.com (Richard Ham),rahdo-talks-through,Rahdo Talks Through,"Richard ""Rahdo"" Ham talks (at great length) al...",4.6,60
5,3cfef94cfccf27eb504bd7a6c7459636,1000035707,CityPodcast,originall,OriginALL,OriginALL este un show cu oameni inteligenti p...,3.7,3
6,2f2205dcc248024b0a3be4665f38e33a,1000068299,PodcastOne,the-big-podcast-with-shaq,The Big Podcast With Shaq,"Much bigger than sports, the show is fun, topi...",4.6,1.3K
8,9fc35a81647e391f93211b5d11973c83,1000081327,"Jim Kettner, John Mihaly, and Scott Brown",galaktacus,Galaktacus,The name Galaktacus originates from a youthful...,5.0,20
14,ce2c3a68dda35600671acd32a1c15fc5,1000095644,Kris Baker Dersch,no-extra-words-one-persons-search-for-story,No Extra Words one person's search for story,A flash fiction podcast featuring short storie...,5.0,11
...,...,...,...,...,...,...,...,...
2077642,075747aa236dd5e97bfa7834a6bce607,999882879,OakGroveBaptist,oakgrovebaptist,OakGroveBaptist,We want to help you make a connection - a conn...,4.0,1
2077650,c108b5f6d6972a6e9d900d222a0e6309,999909601,"Keating Dental Arts: Dental, Dentistry, Dentist",dental-up,Dental Up,Dental Up is your source for everything dental...,5.0,21
2077657,5177efa9361c4af189619dc829590d1c,999952800,New Hampshire Public Radio,the-bookshelf,The Bookshelf,The Bookshelf features authors from around New...,5.0,5
2077659,3c19556d722b83c643f2d0ff59891ec1,999962739,Sarika D. Mehta,sarika-d-mehta-podcast,Sarika D. Mehta » Podcast,Awesome interviews with fascinating people.,5.0,2


In [ ]:
podcasts

In [8]:
categories_df

,podcast_id,itunes_id,category
0,943d1c4349913bde09ef82b69f83cfdf,1446474395,education
1,f53f27d03cdc5040ab4fd2ea30e3dd4f,840646770,society-culture
2,49fa5f1097ae9046ce1a0c56e31b533c,1541959455,arts-performing-arts
3,dbe2dc5aa097523aa294a3b659293af4,1146609744,tv-film
4,2349ed9d26f16029cfc69902bf319ca5,1475252651,education
...,...,...,...
3706363,3cd0ca014c7d0d32dca2b489011e2d9b,1532108170,comedy-comedy-interviews
3706364,6928f41c1daae0d5b56a62a179d27a6b,1440324922,music
3706365,f5da21b82424532981c7b7a1d777249c,1581307262,tv-film-tv-reviews
3706366,4b68b14fdc85128750c6a4275b78ad5b,1521319657,business-entrepreneurship


In [10]:
sorted_cat = categories_df.sort_values(['itunes_id'])
sorted_cat

,podcast_id,itunes_id,category
1865580,d3997f1089be9e45fda4113be39765cb,1000000618,religion-spirituality
3177669,d3997f1089be9e45fda4113be39765cb,1000000618,christianity
2513423,f201172864310534ae6b6c4f8ae9b8cc,1000016089,leisure-games
3183460,f201172864310534ae6b6c4f8ae9b8cc,1000016089,leisure
3622397,4e5ce6a47e5d491aac3f588cfb3ece73,1000016800,christianity
...,...,...,...
1731617,28947e3cd8bb3489e037c5b141b8e92b,999975096,news-business-news
1068771,b863eaa678d3de4f5721a7492fd32b3e,999975096,news-business-news
1175989,920de5c8d1bb6b3fea7d86ac39473d76,999987741,arts
2327110,920de5c8d1bb6b3fea7d86ac39473d76,999987741,arts-books


In [12]:
# Define a custom aggregation function to combine features
def combine_features(series):
    all_features = '-'.join(series)
    unique_features = set(all_features.split('-'))
    return '-'.join(unique_features)

# Group the DataFrame by the first two columns and apply the custom aggregation function
df_cat_grouped = sorted_cat.groupby(['podcast_id', 'itunes_id'])['category'].agg(combine_features).reset_index()


                               podcast_id   itunes_id  \
0        0000160859bbbbdf071b17c440872744  1530837716   
1        000016722a07679f48f29bccd206997e  1523503128   
2        0000169459dbf020240a4e80ab330169  1497332591   
3        0000444ca887adf42236ca9b2ef62c8d  1519019288   
4        000045392ef303398272c094199210a8   315420105   
...                                   ...         ...   
2118877  ffffea74ecd9796d2e8b65844538aad0  1492985653   
2118878  fffff0e15ba4d0db87ec80367e6c32f2  1217677289   
2118879  fffff12ed4a65b4e96e9249d41fea5b7  1504012397   
2118880  fffff6feed44c6156ebbd3c7c2d63a76  1522789533   
2118881  fffffd48a1e983ad2bfdc61d898c1810  1455269804   

                                      category  
0                           investing-business  
1                            basketball-sports  
2           christianity-spirituality-religion  
3                                comedy-improv  
4           christianity-spirituality-religion  
...                  

In [13]:
df_cat_grouped

,podcast_id,itunes_id,category
0,0000160859bbbbdf071b17c440872744,1530837716,investing-business
1,000016722a07679f48f29bccd206997e,1523503128,basketball-sports
2,0000169459dbf020240a4e80ab330169,1497332591,christianity-spirituality-religion
3,0000444ca887adf42236ca9b2ef62c8d,1519019288,comedy-improv
4,000045392ef303398272c094199210a8,315420105,christianity-spirituality-religion
...,...,...,...
2118877,ffffea74ecd9796d2e8b65844538aad0,1492985653,marketing-business
2118878,fffff0e15ba4d0db87ec80367e6c32f2,1217677289,technology-business
2118879,fffff12ed4a65b4e96e9249d41fea5b7,1504012397,history-travel-places-society-culture
2118880,fffff6feed44c6156ebbd3c7c2d63a76,1522789533,business


In [24]:
df = podcasts_limited.merge(df_cat_grouped, on=['podcast_id', 'itunes_id'])
df

,podcast_id,itunes_id,author,slug,title,description,average_rating,ratings_count,category
0,f201172864310534ae6b6c4f8ae9b8cc,1000016089,noreply@blogger.com (Richard Ham),rahdo-talks-through,Rahdo Talks Through,"Richard ""Rahdo"" Ham talks (at great length) al...",4.6,60,games-leisure
1,3cfef94cfccf27eb504bd7a6c7459636,1000035707,CityPodcast,originall,OriginALL,OriginALL este un show cu oameni inteligenti p...,3.7,3,careers-business
2,9fc35a81647e391f93211b5d11973c83,1000081327,"Jim Kettner, John Mihaly, and Scott Brown",galaktacus,Galaktacus,The name Galaktacus originates from a youthful...,5.0,20,film-tv
3,ce2c3a68dda35600671acd32a1c15fc5,1000095644,Kris Baker Dersch,no-extra-words-one-persons-search-for-story,No Extra Words one person's search for story,A flash fiction podcast featuring short storie...,5.0,11,arts-books-performing
4,aed410e50370beec62b241374abb296b,1000114875,Todd Mizener,the-talking-pictures-podcast,The Talking Pictures Podcast,Photojournalism on the radio...sort of. The ph...,5.0,9,visual-news-arts
...,...,...,...,...,...,...,...,...,...
75324,84d95344bd4f9e98afbc36f645817220,999860044,Society of Hospital Medicine,before-the-white-coat,Before The White Coat,"On Before The White Coat, SHM CEO and host Lar...",5.0,2,health-medicine-careers-fitness-business
75325,075747aa236dd5e97bfa7834a6bce607,999882879,OakGroveBaptist,oakgrovebaptist,OakGroveBaptist,We want to help you make a connection - a conn...,4.0,1,christianity-spirituality-religion
75326,c108b5f6d6972a6e9d900d222a0e6309,999909601,"Keating Dental Arts: Dental, Dentistry, Dentist",dental-up,Dental Up,Dental Up is your source for everything dental...,5.0,21,education
75327,3c19556d722b83c643f2d0ff59891ec1,999962739,Sarika D. Mehta,sarika-d-mehta-podcast,Sarika D. Mehta » Podcast,Awesome interviews with fascinating people.,5.0,2,news-arts


In [22]:
def features_to_list(feature_string):
    return feature_string.split('-')
    

In [25]:
df['category'] = df['category'].apply(features_to_list)
df

,podcast_id,itunes_id,author,slug,title,description,average_rating,ratings_count,category
0,f201172864310534ae6b6c4f8ae9b8cc,1000016089,noreply@blogger.com (Richard Ham),rahdo-talks-through,Rahdo Talks Through,"Richard ""Rahdo"" Ham talks (at great length) al...",4.6,60,"[games, leisure]"
1,3cfef94cfccf27eb504bd7a6c7459636,1000035707,CityPodcast,originall,OriginALL,OriginALL este un show cu oameni inteligenti p...,3.7,3,"[careers, business]"
2,9fc35a81647e391f93211b5d11973c83,1000081327,"Jim Kettner, John Mihaly, and Scott Brown",galaktacus,Galaktacus,The name Galaktacus originates from a youthful...,5.0,20,"[film, tv]"
3,ce2c3a68dda35600671acd32a1c15fc5,1000095644,Kris Baker Dersch,no-extra-words-one-persons-search-for-story,No Extra Words one person's search for story,A flash fiction podcast featuring short storie...,5.0,11,"[arts, books, performing]"
4,aed410e50370beec62b241374abb296b,1000114875,Todd Mizener,the-talking-pictures-podcast,The Talking Pictures Podcast,Photojournalism on the radio...sort of. The ph...,5.0,9,"[visual, news, arts]"
...,...,...,...,...,...,...,...,...,...
75324,84d95344bd4f9e98afbc36f645817220,999860044,Society of Hospital Medicine,before-the-white-coat,Before The White Coat,"On Before The White Coat, SHM CEO and host Lar...",5.0,2,"[health, medicine, careers, fitness, business]"
75325,075747aa236dd5e97bfa7834a6bce607,999882879,OakGroveBaptist,oakgrovebaptist,OakGroveBaptist,We want to help you make a connection - a conn...,4.0,1,"[christianity, spirituality, religion]"
75326,c108b5f6d6972a6e9d900d222a0e6309,999909601,"Keating Dental Arts: Dental, Dentistry, Dentist",dental-up,Dental Up,Dental Up is your source for everything dental...,5.0,21,[education]
75327,3c19556d722b83c643f2d0ff59891ec1,999962739,Sarika D. Mehta,sarika-d-mehta-podcast,Sarika D. Mehta » Podcast,Awesome interviews with fascinating people.,5.0,2,"[news, arts]"


In [30]:
cleaned_podcast_df = df
output = open('cleaned_podcast.pkl', 'wb')
pickle.dump(cleaned_podcast_df,output)

In [31]:
with open('data/reviews.json') as f:
    reviews_df = pd.DataFrame(json.loads(line) for line in f)
reviews_df.head()

,podcast_id,title,content,rating,author_id,created_at
0,52e3d2c4fab4e80a8bb75ad144671d96,Argh,What the heck is this show? And why can’t I de...,1,a23655ce5565f32,2019-05-01 01:24:01+00
1,52e3d2c4fab4e80a8bb75ad144671d96,Miss gross overrated as interviewer.,The Howard stern interview was horrible. I lea...,1,e5d461c5f91c8d2,2019-06-07 14:25:34+00
2,52e3d2c4fab4e80a8bb75ad144671d96,I love Terri and Fresh Air,People get really pissy about this beautiful f...,5,806cc75442c8f3d,2019-05-15 09:42:53+00
3,52e3d2c4fab4e80a8bb75ad144671d96,Fresh Air subscription changes to Code Switch,I’ve been a subscriber to Fresh Air for over a...,4,3e1e7e8417a3e85,2019-05-06 02:39:10+00
4,52e3d2c4fab4e80a8bb75ad144671d96,Climate change,"Hello, my name is Lily Rudman and I am a fifth...",5,7c680144b582c6d,2019-04-18 00:37:51+00


In [33]:
reviews_df =reviews_df.rename(columns={'title': 'review_title', 'content': 'review_text'})
reviews_df

,podcast_id,review_title,review_text,rating,author_id,created_at
0,52e3d2c4fab4e80a8bb75ad144671d96,Argh,What the heck is this show? And why can’t I de...,1,a23655ce5565f32,2019-05-01 01:24:01+00
1,52e3d2c4fab4e80a8bb75ad144671d96,Miss gross overrated as interviewer.,The Howard stern interview was horrible. I lea...,1,e5d461c5f91c8d2,2019-06-07 14:25:34+00
2,52e3d2c4fab4e80a8bb75ad144671d96,I love Terri and Fresh Air,People get really pissy about this beautiful f...,5,806cc75442c8f3d,2019-05-15 09:42:53+00
3,52e3d2c4fab4e80a8bb75ad144671d96,Fresh Air subscription changes to Code Switch,I’ve been a subscriber to Fresh Air for over a...,4,3e1e7e8417a3e85,2019-05-06 02:39:10+00
4,52e3d2c4fab4e80a8bb75ad144671d96,Climate change,"Hello, my name is Lily Rudman and I am a fifth...",5,7c680144b582c6d,2019-04-18 00:37:51+00
...,...,...,...,...,...,...
5607016,0c06304da3173d464b4487d6040b46d4,Love!,Alex! You are hitting it out of the park with ...,5,25c5550051cf584,2023-03-03 13:07:43+00
5607017,c1b72d56046b06a69cc083a6a5732051,Simple way to get ahead,The discussion on the role of the Export-Impor...,5,750e3e2492a9c13,2023-03-03 16:20:13+00
5607018,95954ad0693967fa1d87dee305cd6a30,Wonderful podcast; ill-fitting intro song,It made no sense in the context at all. Great ...,4,09c2426888b6a94,2023-03-03 15:36:57+00
5607019,2c86f61c1e26ff641a95f5bff6297dfd,Binging like a madman,"My girlfriend turned me onto this podcast, and...",5,fd53bc0a84bd210,2023-03-03 13:29:11+00


In [34]:
common_ids = set(df['podcast_id']).intersection(set(reviews_df['podcast_id']))
print(len(common_ids))  

27


In [36]:
print(df['podcast_id'].nunique())  # Count unique values in df
print(reviews_df['podcast_id'].nunique())  # Count unique values in reviews_df


75329
303911


In [37]:
df1 = df.copy()

In [45]:
education_df = df1[df1['category'].apply(lambda x: 'education' in x)]
education_df


,podcast_id,itunes_id,author,slug,title,description,average_rating,ratings_count,category
6,7069df204b03df17ae91b4fa92aa3c64,1000211238,James Boulton,queens-of-england-podcast,Queens of England Podcast,Too often when we think about the history of a...,4.9,32,"[culture, society, education]"
10,18d1296dea18679c35d3d6c321edfc25,1000276614,CityPodcast,techvolution,TechVolution,TechVolution este show-ul în care vorbim despr...,5.0,2,"[education, technology]"
11,5a56b74c2818652897d824377a569afb,1000291435,"Joel Lindstrom, Shawn Tabor, George Doubinski,...",crm-audio,CRM Audio,CRM Audio is a network of podcasts about Dynam...,4.1,8,"[education, technology]"
17,bbbbfc39393438d46966dae45d5936d1,1000407917,Paul Sian,cincinnati-northern-kentucky-real-estate,Cincinnati & Northern Kentucky Real Estate,Real estate podcast,5.0,1,[education]
28,d613be5c466e34c10c1df87afcca4353,1000758422,"Soirée-Leone: Gardening & Composting, Canning ...",permaculture-velocity-homesteading-skills-you-...,Permaculture Velocity | Homesteading Skills Yo...,"Edible gardens. Chickens, geese, and ducks. Go...",4.6,26,"[culture, society, education]"
...,...,...,...,...,...,...,...,...,...
75286,bc0a8f4c722c92ed738b25cd6ba15517,998736166,User 219201698,advanced-movers-spring-branch-isd,Advanced Movers Spring Branch ISD,Podcast by User 219201698,4.0,1,[education]
75300,c3dd49c2431964c9e3a5f08b12d4ef25,999262014,Caddo Mills church of Christ,caddo-mills-church-of-christ,Caddo Mills church of Christ,All to often in life we are confronted with th...,5.0,2,"[christianity, spirituality, education, religion]"
75310,963223c55c34d3280f6a233bdb02be6d,999487472,PrepAgent.com,pass-the-real-estate-exam-with-prepagent,Pass the Real Estate Exam with PrepAgent,Passing the real estate exam does not have to ...,4.6,94,"[careers, courses, education, business]"
75323,852678346b67bfd44d10b67ae594e96a,999802464,Carey Green,podcastification-podcasting-tips-podcast-trick...,"Podcastification - podcasting tips, podcast tr...","Podcastificating the world, one listener at a ...",4.9,30,"[how, to, education, technology]"


In [38]:
full_df = df1.merge(reviews_df, on='podcast_id')
full_df

,podcast_id,itunes_id,author,slug,title,description,average_rating,ratings_count,category,review_title,review_text,rating,author_id,created_at
0,03a17efdb4a8a11d64d72000a53690e1,1006403754,Jen Hemphill,her-money-matters,Her Money Matters,"Jen Hemphill, Money Confidence Coach goes on a...",4.8,95,"[education, investing, how, to, business]",Podcast Guest Booker/ Writer,This is one of my favorite shows. It's clear t...,5,29dd6a70bd8b3bc,2017-02-24 22:26:09+00
1,03a17efdb4a8a11d64d72000a53690e1,1006403754,Jen Hemphill,her-money-matters,Her Money Matters,"Jen Hemphill, Money Confidence Coach goes on a...",4.8,95,"[education, investing, how, to, business]",Learning Spanish love it!,Love that you are incorporating spanish! Aweso...,5,883f6252ca1c4ef,2018-04-08 18:46:40+00
2,03a17efdb4a8a11d64d72000a53690e1,1006403754,Jen Hemphill,her-money-matters,Her Money Matters,"Jen Hemphill, Money Confidence Coach goes on a...",4.8,95,"[education, investing, how, to, business]",Great show!,Jen does such a great job of talking about mon...,5,9fc0eed0609943c,2015-08-11 21:32:12+00
3,03a17efdb4a8a11d64d72000a53690e1,1006403754,Jen Hemphill,her-money-matters,Her Money Matters,"Jen Hemphill, Money Confidence Coach goes on a...",4.8,95,"[education, investing, how, to, business]",The Heart of Money Matters,I look forwarrd to Jen's podcast every week! J...,5,96cdf2308511db6,2016-08-02 15:37:05+00
4,03a17efdb4a8a11d64d72000a53690e1,1006403754,Jen Hemphill,her-money-matters,Her Money Matters,"Jen Hemphill, Money Confidence Coach goes on a...",4.8,95,"[education, investing, how, to, business]","Informative, Entertaining Money Talk",I love this podcast about money from Jen Hemph...,5,a5e9e57f6162fee,2015-10-28 14:24:25+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,1e46aafcd91e6e46b1af9932776a002d,640440434,Hank Thorpe,all-of-the-above-radio,All of the Above radio,All of the Above radio: saving the world two h...,5.0,8,"[personal, society, culture, journals]",Great Music and Conversation,This show has a great mix of music with fun to...,5,fb6ca03240db301,2016-11-03 17:17:06+00
277,1e46aafcd91e6e46b1af9932776a002d,640440434,Hank Thorpe,all-of-the-above-radio,All of the Above radio,All of the Above radio: saving the world two h...,5.0,8,"[personal, society, culture, journals]",Full escapism is the truth!,This show is great to take a break from the ca...,5,e3713d1d11c289a,2018-08-19 19:55:01+00
278,1e46aafcd91e6e46b1af9932776a002d,640440434,Hank Thorpe,all-of-the-above-radio,All of the Above radio,All of the Above radio: saving the world two h...,5.0,8,"[personal, society, culture, journals]",Hank’s the Man,I’ve known Hank for close to 20 years. He’s an...,5,cb14669ad97547c,2015-07-14 18:08:30+00
279,1e46aafcd91e6e46b1af9932776a002d,640440434,Hank Thorpe,all-of-the-above-radio,All of the Above radio,All of the Above radio: saving the world two h...,5.0,8,"[personal, society, culture, journals]",Hank is my dude,Great show. Can't be mad at anything while thi...,5,e1e474835a0be76,2018-01-27 16:11:26+00


In [ ]:
reviews_df



,podcast_id,title,content,rating,author_id,created_at
0,52e3d2c4fab4e80a8bb75ad144671d96,Argh,What the heck is this show? And why can’t I de...,1,a23655ce5565f32,2019-05-01 01:24:01+00
1,52e3d2c4fab4e80a8bb75ad144671d96,Miss gross overrated as interviewer.,The Howard stern interview was horrible. I lea...,1,e5d461c5f91c8d2,2019-06-07 14:25:34+00
2,52e3d2c4fab4e80a8bb75ad144671d96,I love Terri and Fresh Air,People get really pissy about this beautiful f...,5,806cc75442c8f3d,2019-05-15 09:42:53+00
3,52e3d2c4fab4e80a8bb75ad144671d96,Fresh Air subscription changes to Code Switch,I’ve been a subscriber to Fresh Air for over a...,4,3e1e7e8417a3e85,2019-05-06 02:39:10+00
4,52e3d2c4fab4e80a8bb75ad144671d96,Climate change,"Hello, my name is Lily Rudman and I am a fifth...",5,7c680144b582c6d,2019-04-18 00:37:51+00
...,...,...,...,...,...,...
5607016,0c06304da3173d464b4487d6040b46d4,Love!,Alex! You are hitting it out of the park with ...,5,25c5550051cf584,2023-03-03 13:07:43+00
5607017,c1b72d56046b06a69cc083a6a5732051,Simple way to get ahead,The discussion on the role of the Export-Impor...,5,750e3e2492a9c13,2023-03-03 16:20:13+00
5607018,95954ad0693967fa1d87dee305cd6a30,Wonderful podcast; ill-fitting intro song,It made no sense in the context at all. Great ...,4,09c2426888b6a94,2023-03-03 15:36:57+00
5607019,2c86f61c1e26ff641a95f5bff6297dfd,Binging like a madman,"My girlfriend turned me onto this podcast, and...",5,fd53bc0a84bd210,2023-03-03 13:29:11+00
